# Text analysis

This notebook contains code for the  analysis of text data used in the project: Public attitudes and ethical guidelines in digital field experiments (digex).

# TO DO

**We can roughly use the below bullet points taken from [2022-digex-study-design](https://docs.google.com/document/d/1nsaXEn04s9LTsjqrpbUpZlyQ3CAcuqkOwy1ZM0d-kKY/edit#) as section headings for this notebook to conduct the exploratory analysis**:

- To further address questions 1-3, open-ended free-text inputs of both the open “other” options of selection items and the  open-ended free-text answers (i.e., what do you think it means for an academic study to receive "ethical approval”, describe any concerns you might have, what additional information about the study or the researchers that would influence your level of concern, are there any other features of research that are important for determining your level of concern, are there any additional factors that you think should be considered) will be analyzed using a mixed-methods approach to detect common topics, sentiments, and themes. 
 
- The mixed-methods analysis approach will consist of combining quantitative textual analysis, specifically, topic (detection) modeling and sentiment analysis, with qualitative coding, specifically, structured tabular thematic analysis (ST-TA). Topic modeling will involve applying Latent Dirichlet Allocation to identify words within each  open-ended free-text response that are most frequently associated with each of the k identified topics, and produce the probability that each open-ended free-text response within the corpus of responses is associated with each of k topics (Jelodar et al. 2019). Sentiment analysis will involve assigning a polarity score in the range [−1, 1] through the Valence Aware Dictionary for sEntiment Reasoning (VADER) method. VADER is a lexicon-based sentiment analysis engine that combines lexicon-based methods with a rule-based modeling consisting of human validated rules Elbagir and Yang (2019). Models will be implemented using the R package topicmodels and the Python packages Natural Language Toolkit (NLTK). 

- The purpose of manual coding is to validate the topic models, specifically, the representatives of the detected topics and the number of topics k, and to identify further themes (Züll 2016). To implement ST-TA we will follow the method described by Robinson (2021). One to two coders will qualitatively identify themes mentioned by the participants and subsequently categorize all responses accordingly in order to receive a frequency measure. Specifically, coders will read the responses and add each theme as a column in a coding sheet. It will be coded whether the respective theme appears in the other responses (= 1) or not (= 0). If a coder encounters new relevant themes, they will be added and coded later. Nonsense responses will be excluded during coding. Ambiguities will be discussed and solved in pairs. If no solution can be found, a third coder will be consulted. Results from the  open-ended free-text items will be displayed in frequency tables or plots, word clouds, and summarized in quotes. 

## Set working directory

In [1]:
import pathlib   # Change cwd
import os 

path = pathlib.Path.cwd().parent
os.chdir(path)

## Imports

In [2]:
import yaml   # 3rd party packages
import warnings
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pandas_profiling import ProfileReport

from digex_src import config    # Local imports
from digex_src import preprocess_data
from digex_src.load_data import get_data_filepath

warnings.filterwarnings('ignore')    # Ignore warnings

## Plotting presets

In [7]:
digex_style = config.MPL_STYLE_FILEPATH
digex_palette = config.PALETTE

plt.style.use(digex_style)
sns.color_palette(digex_palette)

[(0.00392156862745098, 0.09803921568627451, 0.34901960784313724),
 (0.058823529411764705, 0.2235294117647059, 0.37254901960784315),
 (0.08627450980392157, 0.3137254901960784, 0.3843137254901961),
 (0.15294117647058825, 0.38823529411764707, 0.37254901960784315),
 (0.2823529411764706, 0.44313725490196076, 0.30980392156862746),
 (0.42745098039215684, 0.48627450980392156, 0.23137254901960785),
 (0.592156862745098, 0.5333333333333333, 0.17254901960784313),
 (0.7686274509803922, 0.5686274509803921, 0.2196078431372549),
 (0.9215686274509803, 0.6039215686274509, 0.3764705882352941),
 (0.9882352941176471, 0.6627450980392157, 0.5843137254901961),
 (0.9921568627450981, 0.7294117647058823, 0.7803921568627451),
 (0.9803921568627451, 0.8, 0.9803921568627451)]

## Load processed data

In [ ]:
processed_data_path = get_data_filepath(
    file=config.PROCESSED_DATA_FILEPATH, 
    data_path=config.PROCESSED_DATA_DIR,
    main=False
) 

digex_df = pd.read_csv(processed_data_path, index_col=0)

digex_df.head()

,duration_sec,finished,sm_use,age,gender_id,ethnic_id,edu,politic_views,aware_sm_res,aware_sm_advan,...,rank_pub_interst,rank_add_fac_1,rank_add_fac_1_pos,rank_add_fac_2,rank_add_fac_2_pos,rank_add_fac_3,rank_add_fac_3_pos,aware_sm_advan_score,aware_sm_interact_score,aware_sm_use_score
1,912.0,True,Facebook,29.0,Male,Asian - Eastern,Highschool,Slightly liberal,Extremely aware,['… are large and can contain millions of data...,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4,0,9
2,720.0,True,Twitter,33.0,Male,Mixed race,Highschool,Neutral/ Neither conservative or liberal,Moderately aware,['… are large and can contain millions of data...,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,1,1,9
3,1874.0,True,Facebook,33.0,Female,Pacific Islander,Bachelor's degree,Very liberal,Extremely aware,['… are large and can contain millions of data...,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2,2,5
4,1264.0,True,Facebook,73.0,Female,White / Caucasian,Highschool,Slightly conservative,Moderately aware,['… are large and can contain millions of data...,...,1.0,NaN,8.0,NaN,NaN,NaN,NaN,1,1,6
5,556.0,True,Twitter,27.0,Female,Native-American,Highschool,Very liberal,Extremely aware,['… often capture social relationships not fou...,...,7.0,NaN,NaN,NaN,NaN,NaN,NaN,0,3,9


## Exploratory data analysis

Resources:
- see `docs/resources/`
- Text as Data, Chris Bail

**Variables to examine: 15, 17, 18, 20, 21, 23, 24, 26, 27, 37, 45-50**